In [32]:
from scipy.interpolate import CubicSpline
from scipy.interpolate import interp1d
from scipy.misc import derivative
import numpy as np
import csv
import matplotlib.pyplot as plt
import matplotlib.animation as animation

In [ ]:
%matplotlib inline

#This profiling alorithim is based on this paper:
# Motion Planning of SCARA Robots for Trajectory Tracking by Giovanni Incerti


#Does parametric Cubic Spline interpolation and returns the spline and vector coordinates for every multiple of delta from 0 <= t <= 1
def cubicInterpolate(xList, yList, delta):
    xList = np.array(xList)
    yList = np.array(yList)
    sizeX = xList.size
    sizeY = yList.size
    t = np.linspace(0, 1, sizeX)
    r = np.vstack((xList.reshape((1, sizeX)), yList.reshape(1, sizeY)))
    spline = interp1d(t, r, kind='cubic')
    times = np.linspace(np.min(t), np.max(t),int(1/delta))
    r = spline(times)
    return [spline, r]

#Converts (x, y) into (angle1, angle2) of SCARA joints using inverse kinematics
def convertToAngularPosition(x, y, l1, l2):
    angle2 = np.arccos((x**2 + y**2 - l1**2 -l2**2)/(2*l1*l2))
    angle1 = np.arctan2(y, x) - np.arctan2(l2*np.sin(angle2), l1 + l2*(np.cos(angle2)))
    return [angle1, angle2]
    #for index, (xPoint, yPoint) in enumerate(zip(x, y)):

#Converts linear velocity into angular velocity using the inverse of the Jacobian matrix        
def convertToAngularVelocity(linearVelocityVector, jacobianInverse):
    return np.dot(jacobianInverse, linearVelocityVector)

#Converts linear acceleration into angular acceleration using the derivative and inverse of the Jacobian matrix
def convertToAngularAcceleration(linearAccelerationVector, angularVelocityVector,  jacobianInverse, jacobianDerivative):
    return np.dot(jacobianInverse, (linearAccelerationVector - np.dot(jacobianDerivative, angularVelocityVector)))

#This takes the numeric derivative of the spline and returns a velocity vector
#TODO: find a function in some library to do this particular differentiation (make sure you can specify whether to not use the center formula for differentiation as the extremes of the path have no negative parameter t)
def numericDerivativeSpline(parameterizedSpline, number,  delta):
    print(parameterizedSpline(number))
    return (parameterizedSpline(number+delta) - parameterizedSpline(number))/delta

#This takes the second numeric derivative of the spline and returns an acceleration vector
#TODO: find a function in some library to do this particular differentiation (make sure you can specify whether to not use the center formula for differentiation as the extremes of the path have no negative parameter t)
def numericSecondDerivativeSpline(parameterizedSpline, number, deltaT, deltaV):
    return (numericDerivativeSpline(parameterizedSpline, number + deltaV, deltaT) - numericDerivativeSpline(parameterizedSpline, number, deltaT))/deltaV

#Create a motion profile in the form of a 2D array, with each column in the array specifying an angular positon, velocity, or acceleration for every deltaX in the movement
def createProfile(knotPointsX, knotPointsY, delta, l1, l2):
    motionProfile = []
    #First, define the path using the interpolation
    fAndCoords = cubicInterpolate(knotPointsX, knotPointsY, delta)
    #Iterate through the x and y coordinates, specifying which multiple of t you are on.
    for i, (x, y) in enumerate(zip(fAndCoords[1][0,:], fAndCoords[1][1,:])):
        #Convert the cartesian position to angles the arm can go to to reach position
        angles = convertToAngularPosition(x, y, l1, l2)
        #Define J^-1
        jacobianInverse = (1/(l1*l2*np.sin(angles[1]))) * np.array([[l2 * np.cos(sum(angles)), l2 * np.sin(sum(angles))],
                                                                    [-l1 * np.cos(angles[0]) - l2 * np.cos(sum(angles)), -l1 * np.sin(angles[0]) - l2 * np.sin(sum(angles))]])
        #Caculate the angular velocity vector based on the cartesian velocity vector of the interpolation
        anglesVelocity = np.vstack(convertToAngularVelocity(numericDerivativeSpline(fAndCoords[0],i*delta,1e-6), jacobianInverse))
        #Define J'
        jacobianDerivative = np.array([[-l1*anglesVelocity[0][0]*np.cos(angles[0]) - sum(anglesVelocity)[0]*l2*np.cos(sum(angles)), -l2*sum(anglesVelocity)[0]*np.cos(sum(angles))],
                                       [-l1*anglesVelocity[0][0]*np.sin(angles[0]) - sum(anglesVelocity)[0]*l2*np.sin(sum(angles)), -l2*sum(anglesVelocity)[0]*np.sin(sum(angles))]])
        #Caculate the angular acceleration vector based on the cartesian acceleration vector of the interpolation
        anglesAcceleration = np.vstack(convertToAngularAcceleration(numericSecondDerivativeSpline(fAndCoords[0],i*delta,1e-6, 1e-6), anglesVelocity, jacobianInverse, jacobianDerivative))
        #Define one line of the profile in the format (angle1, angle2, angle1Velocity, angle2Velocity, angle1Acceleration, angle2Acceleration)
        profileComponent = ([angles[0], angles[1], anglesVelocity[0], anglesVelocity[1], anglesAcceleration[0], anglesAcceleration[1]])
        #Debug
        print(profileComponent)
        #Add the line to the profile
        motionProfile.append(profileComponent)
    #Return profile
    return motionProfile

#Convert the motion profile generated from the above funtion from being measured in radians to the position being meausred in centidegrees, and the other two values being measured in degrees
def convertToDegreeSystem(profileComponent):
    return [profileComponent[0] * (18000/np.pi), profileComponent[1] * (18000/np.pi), profileComponent[2][0] * (180/np.pi), profileComponent[3][0] * (180/np.pi), profileComponent[4][0] * (180/np.pi), profileComponent[5][0] * (180/np.pi)]

#Convert the motion profile generated from createProfile from being measured in radians starting at the x axis to being meausured in the robot's local coordinate system
def convertToRobotCoordinateSystem(profileComponent):
    return convertToDegreeSystem([-(profileComponent[0]) + np.pi/2, -(profileComponent[1]), -profileComponent[2], -profileComponent[3], -profileComponent[4], -profileComponent[5]])




knotX = [0, 2, 4, 2, 0, -2, -4, -2, 0]
knotY = [5, 7, 5, 3, 5, 7, 5, 3, 5]

#Creates a profile and animates it.
arm1Length = 10
arm2Length = 10
profile = createProfile(knotX, knotY, 0.001, arm1Length, arm2Length)
fig,ax = plt.subplots(figsize=(8, 8))
ax.plot(cubicInterpolate(knotX, knotY, 0.001)[1][0,:], cubicInterpolate(knotX, knotY, 0.001)[1][1,:])
ax.set_xlim(-20, 20)
ax.set_ylim(-20, 20)
u1 = arm1Length*np.cos(profile[0][0])
v1 = arm1Length*np.sin(profile[0][0])
u2 = arm2Length*np.cos(profile[0][0] + profile[0][1])
v2 = arm2Length*np.sin(profile[0][0] + profile[0][1])
arm1 = ax.quiver(0, 0, u1, v1, color='red',units='xy', angles='xy', scale_units='xy', scale=1.)
arm2 = ax.quiver(u1, v1, u2, v2, color='blue',units='xy', angles='xy', scale_units='xy', scale=1.)
frameNumber = ax.text(0.05, 0.95,'',horizontalalignment='left',verticalalignment='top', transform=ax.transAxes)
def animate(num, arm1, arm2):
    u1 = arm1Length*np.cos(profile[num][0])
    v1 = arm1Length*np.sin(profile[num][0])
    u2 = arm2Length*np.cos(profile[num][0] + profile[num][1])
    v2 = arm2Length*np.sin(profile[num][0] + profile[num][1])
    arm1.set_UVC(u1, v1)
    arm2.set_UVC(u2, v2)
    arm2.set_offsets(np.array([u1, v1]).T)
    frameNumber.set_text(num)
    return [arm1, arm2]

anim = animation.FuncAnimation(fig, animate, fargs=(arm1, arm2),frames=len(profile), interval=50, blit=False)
anim.save("arm.gif")
plt.show()


#This function exports a profile that is readable by the SCARA arm into a csv file
#TODO: scale down velocities and accelerations to some preset maximum
def exportProfile(profile, profileName):
    with open(profileName + ".csv", "w+") as file:
        writer = csv.writer(file, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
        for i in profile:
            writer.writerow(convertToRobotCoordinateSystem(i))
exportProfile(profile, "myFirstProfile")

